In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('USvideos.csv')

In [3]:
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [4]:
df.isna().sum()

video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
thumbnail_link              0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
description               570
dtype: int64

In [5]:
df.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [6]:
df_no_na = df.copy().drop(['comments_disabled', 'ratings_disabled','video_error_or_removed'], axis=1)

In [7]:
import datetime
df_no_na['publish_time'] =  pd.to_datetime(df_no_na['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

In [8]:
df_no_na['trending_date'] =  pd.to_datetime(df_no_na['trending_date'], format='%y.%d.%m')

In [9]:
df_after_17 = df_no_na[df_no_na['publish_time'].dt.year >2017]

In [10]:
df_final = df_after_17[(df_after_17['trending_date'] >= '2018-01-13') & (df_after_17['trending_date'] <= '2018-03-31')]

In [11]:
ids = df_final.video_id.unique()

In [12]:
to_be_removed =[]
for curr_id in ids:
    cur = df_final[df_final['video_id']==curr_id]
    for index, row in cur.iterrows():
        if str(row['trending_date']) == '2018-05-01 00:00:00':
            to_be_removed+= [curr_id]
            break    

In [13]:
len(df_final)

15430

In [14]:
for curr_id in to_be_removed:
    df_final = df_final[df_final['video_id']!= curr_id]

In [15]:
len(df_final)

15430

In [16]:
df_final.to_csv("US_final.csv",index=False,sep="\\")

In [17]:
df_us = pd.read_csv("US_final.csv",sep="\\")

In [18]:
# add url link column
df_us["video_url"] = "https://www.youtube.com/watch?v="+df_us["video_id"]

In [19]:
#clean tags
df_tags_cleaned = df_us.copy()
new_tags = []
for index, row in df_us.iterrows():
    temp_tags= '|'.join([tag.strip().strip('"').strip('“').strip('”').strip() for tag in df_us.iloc[index]["tags"].split("|")])
    new_tags += [temp_tags]
df_tags_cleaned["tags"] = new_tags

In [20]:
df_tags_cleaned = df_tags_cleaned[df_tags_cleaned["tags"]!="[none]"]

In [21]:
#add colunmns for category name and category cluster
cate_map = {43 : "Entertainment",29 : "Nonprofits & Activism",28 : "Science & Technology",27 : "Education",26 : "Howto & Style",25 : "News & Politics",24 : "Entertainment",23 : "Comedy",22 : "People & Blogs",20 : "Gaming",19 : "Travel & Events",17 : "Sports",15 : "Pets & Animals",10 : "Music",2 : "Autos & Vehicles",1 : "Film & Animation"}
cluster_map = {43 : "Entertainment",29 : "Activism",28 : "Technology",27 : "Education",26 : "Lifestyle",25 : "News",24 : "Entertainment",23 : "Entertainment",22 
: "Lifestyle",20 : "Entertainment",19 : "Lifestyle",17 : "Entertainment",15 : "Entertainment",10 : "Entertainment",2 : "Lifestyle",1 : "Entertainment"}

df_with_cate = df_tags_cleaned.copy()
df_with_cate['category_name']="none"
df_with_cate['cluster']="none"

for index, row in df_with_cate.iterrows():
    df_with_cate.at[index,"category_name"] = cate_map[df_with_cate.at[index,"category_id"]]
    df_with_cate.at[index,"cluster"] = cluster_map[df_with_cate.at[index,"category_id"]] 



In [22]:
#clean date format
import datetime
import math
def days_taken_to_trend(start_time,end_time):
    def to_datetime(date_str):
        return datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
#     start = to_datetime(df_tags_cleaned.iloc[0]["publish_time"])
    start = to_datetime(publish_time)
    end = to_datetime(trending_time)

    delta_days = (end-start if end>start else datetime.timedelta(days=1)-(start-end)).total_seconds()/(3600*24)
    return delta_days

In [23]:
#calculate temperature
#temp_raw_score = views*0.2+likes*0.3+cemment_count*0.5
#if not first day in trending : temp_score = (temp_raw_score of today - temp_raw_score of prev day)/ delta_day
#if first day in trending : temp_score = (temp_raw_score of today)/ delta_day since publish

unique_videos= df_with_cate.video_id.unique()
df_with_temp = df_with_cate.copy()
df_with_temp['temperature'] = 0
# print(unique_videos[:2])
for curr_vid in unique_videos:
    curr_df = df_with_temp[df_with_temp["video_id"]==curr_vid]
    i = 0
    prev_index = 0
    prev_raw_score = 0
    for index, row in curr_df.iterrows():
#         curr_raw_score = row["views"]*0.2+row["likes"]*0.3+row["comment_count"]*0.5
        curr_raw_score = row["views"]
        if i==0: #first day
            publish_time = row["publish_time"]
            trending_time = row["trending_date"]
            delta_days = float(days_taken_to_trend(publish_time,trending_time))
            curr_temp = curr_raw_score/delta_days
        else:
            start_time = curr_df.loc[prev_index]["trending_date"]
            end_time = curr_df.loc[index]["trending_date"]
            delta_days = math.ceil(days_taken_to_trend(start_time,end_time))
            curr_temp = (curr_raw_score-prev_raw_score)/delta_days
            if curr_raw_score < prev_raw_score:
                curr_temp = curr_raw_score / delta_days
            
        df_with_temp.at[index,"temperature"] = round(curr_temp,2)
        
        i+=1
        prev_index = index
        prev_raw_score = curr_raw_score
    

In [82]:
#scale temperature down to [0~100]
df_with_temp_scaled = df_with_temp.copy()
factor = 0.5
# temp_range = math.log(max(df_with_temp["temperature"])-min(df_with_temp["temperature"]))
temp_range = math.pow(max(df_with_temp["temperature"])-min(df_with_temp["temperature"]),factor)
for index, row in df_with_temp.iterrows():
    try:
        temp = math.pow(df_with_temp.at[index,"temperature"],factor)/temp_range*100
        df_with_temp_scaled.at[index,"temperature"] = temp+1 if temp<50 else temp
    except:
        df_with_temp_scaled.at[index,"temperature"] = math.pow(0.001,factor)/temp_range*100

In [83]:
#like:dislike ratio
df_ratio = df_with_temp_scaled.copy()
df_ratio["ratio"] = 0
for i in df_ratio.index:
    ratio = df_ratio.at[i,"likes"]/(df_ratio.at[i,"likes"]+df_ratio.at[i,"dislikes"]) if (df_ratio.at[i,"likes"]+df_ratio.at[i,"dislikes"])!=0 else 0.5
    
    ratio*=100
    print(round(ratio,2))
    df_ratio.at[i,"ratio"] = round(ratio,2)

95.7
82.77
88.39
93.18
98.26
92.41
97.04
95.13
97.56
98.36
89.04
93.63
84.42
93.91
95.7
94.21
95.69
99.05
98.05
98.83
99.26
98.34
88.57
91.93
99.3
95.08
91.76
98.38
67.85
97.87
98.13
99.53
80.45
99.36
97.69
99.01
89.18
95.7
99.27
80.82
97.8
98.44
98.02
98.24
52.92
87.56
85.8
98.49
99.02
94.35
90.84
98.79
98.1
97.07
93.07
92.18
98.22
91.91
89.11
98.98
82.28
96.23
99.88
66.5
96.73
94.43
99.13
96.7
95.85
99.74
98.38
97.87
92.9
89.74
99.04
98.73
91.54
92.06
99.21
87.09
90.12
97.36
98.07
99.67
97.92
97.1
99.36
89.8
98.36
95.06
96.51
99.36
97.63
96.51
99.17
97.4
99.04
99.4
98.74
98.65
98.94
98.41
98.17
97.92
68.75
89.39
88.0
99.45
98.64
97.06
94.82
91.21
98.69
98.39
77.95
88.12
99.02
97.4
98.87
97.83
98.24
94.97
89.43
98.15
98.38
93.26
96.71
96.5
67.71
66.74
97.32
98.3
99.01
91.81
47.95
89.77
98.66
94.79
96.11
95.56
97.05
99.05
95.52
94.75
99.33
96.09
25.4
97.12
95.83
96.2
95.77
91.94
98.16
76.56
42.53
93.84
98.0
97.09
99.12
98.34
98.78
95.78
98.08
89.1
33.48
93.75
84.83
96.24
92.79
97.82
95

96.58
99.16
50.0
93.91
92.54
99.06
96.31
98.16
99.36
95.1
92.08
89.82
97.58
86.61
84.2
97.67
99.18
92.56
92.65
98.02
98.19
98.76
97.5
94.07
99.62
92.49
88.78
89.54
92.66
98.62
85.81
94.43
99.26
95.81
98.93
98.63
94.48
72.54
96.97
79.03
98.33
99.65
96.43
97.65
98.9
98.69
99.03
100.0
98.48
23.32
98.62
85.14
99.26
97.66
32.56
73.72
50.0
98.76
98.65
92.47
95.0
99.04
99.63
99.29
96.09
97.3
48.32
98.96
96.74
95.91
84.35
99.24
93.7
98.27
97.64
98.95
90.63
97.2
56.14
93.53
97.22
90.56
99.75
84.69
95.57
98.38
77.57
96.52
99.18
50.0
91.71
97.62
76.91
99.21
71.63
94.58
73.92
22.95
98.02
76.19
90.7
97.04
42.12
72.48
70.56
100.0
96.61
94.15
93.31
98.42
99.06
88.13
93.37
33.33
96.84
95.04
81.68
98.75
100.0
98.14
99.23
80.62
98.97
97.53
82.85
92.06
91.67
17.04
98.57
96.11
91.42
98.99
96.88
98.37
99.05
98.18
91.76
78.57
99.09
91.45
91.2
99.22
99.45
89.94
100.0
98.61
98.61
98.11
94.58
98.39
35.8
82.26
87.26
96.11
94.56
96.89
97.66
93.71
98.86
95.98
92.22
91.31
87.34
98.13
97.36
98.78
99.03
91.37
98.65


98.17
92.71
96.41
72.9
89.95
96.09
93.56
92.97
94.01
93.87
89.29
87.28
81.53
98.56
98.53
98.86
98.01
98.53
98.9
79.46
72.29
96.87
85.43
97.13
88.41
70.38
96.82
30.38
87.68
85.59
89.8
93.87
96.02
97.32
96.47
97.14
97.88
94.73
97.32
98.98
98.96
96.77
96.35
98.0
55.01
98.84
96.95
96.3
90.07
95.72
93.84
93.86
94.9
81.25
99.18
98.33
50.0
91.29
97.38
95.97
91.73
74.14
77.09
93.66
69.88
55.51
93.64
63.94
87.74
92.65
97.36
97.96
97.89
91.95
97.96
95.17
98.42
75.65
98.42
98.24
98.72
98.5
95.92
100.0
89.33
97.28
99.11
92.16
98.43
97.6
95.89
96.87
99.54
98.74
99.46
94.87
87.12
82.1
93.26
97.88
100.0
98.86
94.92
98.82
99.29
99.06
96.45
98.51
91.67
98.66
97.49
96.49
97.04
88.46
97.05
99.05
97.08
95.9
97.86
98.47
89.13
93.85
96.09
70.04
88.66
95.02
97.25
62.8
94.43
99.53
97.4
88.4
81.0
97.84
98.35
63.96
94.55
91.6
89.45
46.13
98.7
99.34
87.5
73.58
99.21
98.37
98.04
76.49
97.25
86.82
93.62
86.3
84.08
95.02
98.93
98.97
90.59
98.75
95.7
98.69
96.27
98.76
46.24
50.0
94.11
97.65
76.73
97.76
99.39
98.34
9

84.88
85.95
96.03
86.85
96.69
56.76
61.85
95.58
92.97
98.71
94.44
65.32
96.76
98.95
97.44
98.39
92.91
89.44
97.59
97.69
97.64
98.11
94.21
97.75
97.48
98.72
89.41
92.13
87.4
94.71
97.25
94.04
97.4
88.58
91.17
21.49
95.21
98.0
97.67
98.76
95.94
98.35
97.76
98.77
85.5
96.82
98.23
69.08
70.3
97.32
96.83
98.9
95.85
95.52
95.77
83.27
99.32
98.36
97.23
83.52
98.57
99.27
97.08
96.94
97.69
95.8
97.3
98.92
98.91
99.1
92.39
96.58
97.94
99.25
95.64
98.36
98.86
56.26
98.4
98.96
97.18
99.19
96.76
98.86
98.21
97.56
94.55
99.01
98.96
99.58
94.23
97.15
98.35
94.09
99.03
91.57
45.14
87.47
95.13
90.93
79.98
98.07
90.09
90.03
96.75
81.22
94.47
99.48
96.23
98.1
97.74
96.84
93.57
98.97
97.62
83.84
93.63
98.51
58.89
98.87
31.54
96.71
98.65
98.8
98.1
50.0
98.09
97.8
99.47
98.5
94.38
53.2
97.31
97.69
95.59
90.52
92.97
99.48
98.04
99.36
96.17
92.3
97.31
97.43
77.78
89.89
93.49
97.04
98.33
95.9
96.8
95.98
93.4
69.66
94.97
97.32
98.14
98.86
74.83
50.0
76.88
98.5
96.69
89.8
94.28
95.21
96.19
84.25
28.0
91.91
96.74

98.38
98.81
94.7
97.5
93.2
96.5
95.84
83.63
99.08
98.73
96.31
98.12
98.84
96.02
98.47
98.12
97.38
82.55
81.08
96.29
93.9
93.79
98.31
90.32
96.67
97.44
97.87
97.1
97.69
95.85
97.06
99.28
98.51
95.94
93.86
97.92
96.35
98.8
97.64
98.64
81.34
99.46
95.1
95.17
99.51
95.69
98.3
94.82
97.4
34.59
98.77
21.93
84.51
98.2
98.44
73.75
95.06
98.41
75.92
98.85
86.33
97.99
85.71
92.97
98.47
89.49
81.89
96.01
94.74
89.47
98.18
95.9
44.67
92.98
96.69
72.13
94.36
98.37
97.43
99.41
95.18
35.02
68.21
98.3
80.1
97.33
99.01
92.33
93.5
50.0
99.17
90.61
96.53
93.3
97.73
98.75
98.09
97.87
80.85
98.04
88.75
92.15
95.43
98.88
99.2
97.86
59.33
97.5
96.12
97.42
89.15
98.54
24.78
96.05
98.15
98.96
94.98
88.93
70.56
96.6
96.81
98.48
97.71
91.0
98.13
94.83
81.42
93.77
99.63
98.31
96.92
96.32
97.59
96.23
88.84
96.82
74.96
91.0
95.45
99.16
96.73
98.33
96.35
95.06
94.78
97.14
96.32
98.75
97.14
97.91
98.68
96.21
96.55
94.01
95.85
98.54
98.32
98.38
96.81
98.95
96.73
96.37
92.34
94.09
96.13
100.0
98.19
22.27
96.84
96.63
84

97.93
97.95
96.83
98.21
98.9
91.72
91.13
97.61
96.4
92.86
96.5
97.21
96.42
98.65
97.72
98.64
99.0
98.82
99.37
95.38
98.7
99.12
92.17
39.84
94.37
98.85
82.67
81.84
91.95
75.36
96.3
91.95
93.9
95.49
95.4
88.43
99.12
84.49
70.69
95.04
99.36
98.96
97.28
98.66
97.33
96.64
45.77
97.37
98.8
94.76
98.8
95.76
96.89
98.03
99.08
74.04
94.75
96.35
96.43
96.83
94.55
99.24
96.97
82.07
98.17
92.51
86.28
92.32
96.18
95.13
83.45
97.95
98.51
50.22
97.53
96.55
93.42
92.88
98.17
89.77
98.87
98.04
98.08
95.37
98.26
98.45
94.93
99.67
90.94
98.92
48.18
98.7
53.51
90.59
94.6
98.87
97.57
93.7
94.02
88.27
95.84
94.47
98.8
80.85
98.93
97.91
98.5
98.77
86.36
98.66
60.74
99.53
96.05
87.52
97.53
97.22
98.71
90.23
75.83
98.61
83.66
93.99
89.82
94.82
90.18
98.76
96.49
99.25
97.37
98.51
98.09
99.36
97.36
98.87
97.73
99.56
98.67
97.07
96.38
90.32
97.55
98.43
97.35
96.3
94.87
97.68
98.55
98.49
97.55
99.59
95.34
99.44
98.33
96.22
98.4
93.8
96.32
61.67
86.46
96.84
96.51
97.67
92.84
93.69
95.74
97.92
96.19
98.97
99.09
95.1

97.23
78.91
98.87
98.23
94.13
95.52
93.77
50.0
99.34
82.02
91.13
99.71
55.53
92.54
99.46
98.36
83.09
99.12
98.93
96.02
98.58
95.46
99.2
90.12
95.6
99.23
99.14
97.9
97.03
96.47
95.49
94.44
95.78
93.78
97.34
86.24
96.07
97.91
96.78
98.22
99.17
97.21
96.38
95.73
91.71
97.29
99.14
98.1
95.14
97.84
98.02
96.18
97.76
91.69
97.92
98.17
92.88
99.07
96.29
96.22
98.68
98.94
97.62
98.22
97.1
92.71
41.74
98.84
84.15
96.55
86.75
93.04
93.61
95.31
99.15
97.5
98.69
98.84
95.75
99.1
96.48
94.95
97.44
98.4
98.84
90.98
97.59
77.68
99.07
98.88
97.81
96.22
88.08
97.24
92.75
93.87
94.13
96.35
97.17
99.2
90.07
96.83
97.76
96.81
96.36
98.4
98.53
93.3
97.35
91.94
97.57
91.03
97.38
96.06
96.41
96.1
92.71
94.91
96.88
97.53
98.22
96.87
93.41
95.47
98.35
97.54
96.82
96.19
96.44
97.77
97.31
61.01
98.17
93.64
92.28
96.69
98.33
97.75
95.66
96.88
95.59
99.01
96.34
99.41
97.54
98.4
82.86
97.79
61.92
50.0
97.63
97.97
98.38
97.25
94.62
98.27
99.0
99.44
95.52
97.29
98.54
67.78
97.19
94.72
96.83
99.17
98.29
97.99
94.31
94

96.66
98.73
99.21
97.53
95.36
98.83
93.7
92.59
93.98
85.11
98.86
96.79
98.62
97.54
99.12
96.81
92.96
97.74
97.63
98.27
94.9
97.8
85.78
98.12
96.69
98.1
97.57
98.32
98.26
97.89
89.22
97.9
96.35
99.24
98.6
90.1
97.45
99.54
93.32
98.06
93.23
99.22
96.98
75.41
85.01
97.71
96.89
98.54
97.56
96.25
95.35
95.84
97.12
99.17
95.78
97.78
98.68
99.55
97.67
97.38
96.6
83.34
97.22
98.16
98.68
95.96
84.19
99.64
98.92
92.86
99.13
99.07
96.72
94.56
99.17
87.02
95.98
99.2
93.21
88.39
92.84
97.62
92.56
99.36
96.46
97.91
97.37
89.64
90.23
97.79
97.68
98.99
96.63
89.61
96.76
98.2
97.65
97.93
98.45
97.08
97.9
93.4
91.99
94.87
87.24
96.26
96.91
98.15
97.51
96.7
97.09
94.33
77.94
98.63
96.78
97.22
97.25
95.61
98.93
93.07
86.35
98.7
97.98
96.74
95.21
94.77
98.02
88.97
97.0
97.77
93.92
94.45
97.04
90.76
99.52
99.29
96.16
97.89
98.21
99.12
82.73
96.16
96.82
96.18
98.09
90.17
97.3
95.52
96.83
98.22
96.17
97.83
98.11
97.72
62.04
96.7
98.31
93.86
97.23
96.19
99.41
84.69
98.41
97.59
89.79
98.1
84.76
99.38
67.79
98.5

98.79
91.81
96.0
97.05
85.07
99.04
96.57
99.21
98.29
95.16
95.99
79.99
93.21
97.96
97.94
98.33
99.03
97.01
96.06
96.08
96.45
99.15
97.05
98.72
99.18
92.62
94.06
96.71
84.55
98.95
95.3
98.05
96.65
99.39
98.13
97.78
86.69
96.71
98.38
98.81
99.14
98.33
97.44
98.95
97.02
93.02
87.65
98.08
98.5
76.45
97.72
97.54
96.09
99.17
96.97
98.57
97.77
99.56
97.53
97.32
93.92
96.63
98.11
91.04
98.64
98.54
93.75
93.32
91.84
98.44
88.28
97.67
92.11
97.32
97.46
99.07
99.41
96.98
99.4
99.24
97.81
95.94
98.12
98.17
95.41
96.75
99.29
95.37
95.88
98.65
76.43
98.36
98.49
98.8
98.42
99.18
98.23
96.57
98.78
98.08
99.17
95.78
93.53
90.19
98.43
97.61
95.46
98.8
98.58
94.54
90.28
98.94
97.4
97.42
98.56
38.67
93.63
99.46
99.09
96.57
81.6
98.92
86.57
51.59
95.99
95.07
97.05
98.51
97.58
90.23
96.02
87.11
96.77
87.37
99.08
92.25
98.17
98.25
89.82
97.57
93.11
84.93
99.3
95.65
96.96
97.58
97.15
98.16
99.15
97.05
73.9
89.52
98.7
91.1
93.05
93.12
98.35
96.52
97.03
98.48
97.21
81.4
50.0
95.62
98.37
95.64
99.06
90.5
92.72
9

In [84]:
#calculate trending length
df_trending_length = df_ratio.copy()
df_trending_length["trending_length"] = "" 
id_date_map = {}
for vid in unique_videos:
    mindate = df_trending_length[df_trending_length["video_id"]==vid]["trending_date"].min()
    id_date_map[vid] = mindate
for i in df_trending_length.index:
    days = pd.to_datetime(df_trending_length.at[i,"trending_date"])-pd.to_datetime(id_date_map[df_trending_length.at[i,"video_id"]])
    df_trending_length.at[i,"trending_length"] = int(str(days).split(' ')[0])+1

In [85]:
df_trending_length["category_name"].unique()

array(['Music', 'Comedy', 'Science & Technology', 'Gaming',
       'Entertainment', 'People & Blogs', 'Film & Animation',
       'News & Politics', 'Education', 'Howto & Style', 'Pets & Animals',
       'Sports', 'Travel & Events', 'Autos & Vehicles',
       'Nonprofits & Activism'], dtype=object)

In [86]:
#calculate trending length
money_map = {"Entertainment":2.5,"Music":2.5,"Gaming":6,"Education":3.9,'Sports':3,'News & Politics':2,
            'People & Blogs':3.4,'Howto & Style':4.5,
       'Science & Technology':3.9, 'Pets & Animals':3.9, 'Comedy':2.5,
       'Travel & Events':3.2, 'Film & Animation':3.6, 'Nonprofits & Activism':2.6,
       'Autos & Vehicles':3}
df_money = df_trending_length.copy()
df_money['income'] =0
for i in df_money.index:
     df_money.at[i,'income'] = (df_money.at[i,"views"]+df_money.at[i,"likes"]+df_money.at[i,"comment_count"]-df_money.at[i,"dislikes"])/1000*money_map[df_money.at[i,"category_name"]]

In [87]:
df_money.to_csv("US_final.csv",index=False,sep="\\")

In [88]:
df_money["ratio"].max()

100